In [ ]:
# Instalar bibliotecas no Colab
!pip install tensorflow opencv-python beautifulsoup4 requests selenium googletrans==4.0.0-rc1 --quiet --quiet

import warnings
warnings.filterwarnings("ignore")  # Silenciar warnings

# Importar bibliotecas
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np
import cv2
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
from googletrans import Translator

# Inicializar tradutor Google Translate
translator = Translator()

In [ ]:
# Função para carregar e preparar imagem a partir de arquivo ou URL
def preprocess_image(image_path_or_url):
    if image_path_or_url.startswith('http'):  # Verifica se é uma URL
        response = requests.get(image_path_or_url)
        image = Image.open(BytesIO(response.content))
        image = image.convert('RGB')
        image = np.array(image)
    else:  # Caso contrário, trata como um caminho local
        image = cv2.imread(image_path_or_url)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = cv2.resize(image, (224, 224))
    image = preprocess_input(image)
    return np.expand_dims(image, axis=0)

In [ ]:
# Carregar modelo MobileNetV2
model = MobileNetV2(weights='imagenet')
# Identificar objeto na imagem
def identify_image(image_path_or_url):
    # Processa a imagem
    image = preprocess_image(image_path_or_url)

    # Faz a previsão usando o modelo
    predictions = model.predict(image)

    # Decodifica as previsões
    decoded = decode_predictions(predictions, top=3)[0]

    # Exibe as previsões
    for i, (imagenet_id, label, score) in enumerate(decoded):
        print(f"{i+1}: {label} ({score * 100:.2f}%)")

    # Tradução com Google Translate
    try:
        if decoded and decoded[0][1]:
            original_label = decoded[0][1].replace('_', ' ')  # Substitui _ por espaço
            translated_label = translator.translate(original_label, src='en', dest='pt').text
        else:
            translated_label = "Rótulo não disponível para tradução"
    except Exception as e:
        print(f"Erro na tradução, usando o nome original em inglês: {e}")
        translated_label = decoded[0][1].replace('_', ' ')  # Usa o nome original com _ substituído por espaço

    return translated_label  # Retorna o label traduzido ou o nome original

In [ ]:
# Função para buscar o menor preço
def search_product_online(product_name):
    search_url = f"https://www.google.com/search?q={product_name.replace(' ', '+')}+menor+preço&cr=countryBR"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Coletar informações de produtos
    products = []
    for g in soup.find_all('div', class_='tF2Cxc'):
        title = g.find('h3').text if g.find('h3') else 'N/A'
        link = g.find('a')['href'] if g.find('a') else 'N/A'
        snippet = g.find('span', class_='aCOpRe').text if g.find('span', class_='aCOpRe') else 'N/A'
        products.append((title, link, snippet))

    # Exibir resultados
    for i, (title, link, snippet) in enumerate(products[:5]):
        print(f"\nResultado {i+1}:")
        print(f"Nome: {title}")
        print(f"Link: {link}")
        print(f"Descrição: {snippet}")

In [ ]:
# Fluxo principal
if __name__ == "__main__":
    # Substituir pelo caminho ou URL da sua imagem
    image_path_or_url = "https://a-static.mlcdn.com.br/800x560/computador-completo-intel-i5-4-geracao-8gb-hd-1tb-facil-computadores/olistplus/opmk9w8zulz00ecr/6d80bf9e93aee8114455aa6b0564c6a0.jpeg"  # Ou URL como "https://example.com/imagem.jpg"
    product_name = identify_image(image_path_or_url)
    print(f"Produto identificado (traduzido): {product_name}")
    print("\nBuscando pelo menor preço...")
    search_product_online(product_name)